# #ass supervised1

In [12]:
import numpy as np
import struct

In [13]:

def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows * cols)
        images = images.astype(np.float64) / 255.0  # normalize to [0, 1]
        return images


In [14]:
def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        magic, num_labels = struct.unpack('>II', f.read(8))
        labels = np.fromfile(f, dtype=np.uint8)
        return labels


# Load the MNIST dataset

In [15]:

X = load_mnist_images('train-images.idx3-ubyte')
y = load_mnist_labels('train-labels.idx1-ubyte')


# Subset the dataset to use only class 0 and class 1

In [16]:

idx = np.where((y == 0) | (y == 1))[0]
X = X[idx]
y = y[idx]

# Standardize the dataset

In [17]:

X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_std[X_std == 0] = 0.00000000000000001  # avoid division by zero
X = (X - X_mean) / X_std


# Add a column of ones to X for the bias term

In [18]:

X = np.hstack((np.ones((len(X), 1)), X))

# Set up k-fold cross-validation

In [19]:

k = 10
fold_size = len(X) // k
fold_indices = np.arange(len(X))
np.random.shuffle(fold_indices)

# Set up hyperparameters

In [20]:

learning_rates = [0.01, 0.1, 0.5, 1.0]
num_iterations = 1000

# Perform k-fold cross-validation

In [21]:

for learning_rate in learning_rates:
    accs = []
    for fold in range(k):
   
        start = fold * fold_size
        end = start + fold_size
        X_val_fold = X[fold_indices[start:end], :]
        y_val_fold = y[fold_indices[start:end]]
        X_train_fold = np.delete(X, fold_indices[start:end], axis=0)
        y_train_fold = np.delete(y, fold_indices[start:end])

        
        w = np.zeros(X_train_fold.shape[1])

     
        for i in range(num_iterations):
           
            y_pred = 1 / (1 + np.exp(-np.dot(X_train_fold, w)))

           
            grad = np.dot(X_train_fold.T, (y_pred - y_train_fold)) / len(y_train_fold)

  
            w -=learning_rate * grad

       
        y_pred_val = np.round(1 / (1 + np.exp(-np.dot(X_val_fold, w))))
        acc = np.mean(y_pred_val == y_val_fold)
        accs.append(acc)

    print('Learning rate:', learning_rate)
    print('Average accuracy:', np.mean(accs))


Learning rate: 0.01
Average accuracy: 0.998341232227488
Learning rate: 0.1
Average accuracy: 0.9988941548183254
Learning rate: 0.5
Average accuracy: 0.9988151658767771
Learning rate: 1.0
Average accuracy: 0.998736176935229
